# 실습 내용 : GAN MNIST 실습

 본 실습은 DCGAN(Deep Convolutional Generative Adversarial Network)를 기반으로 하는 GAN모델을 이용해, MNIST 데이터를 생성해보려 한다. 또한 해당 코드는 PyTorch Tutorial을 기반으로 함을 사전에 언급한다.

## DCGAN 실습

### - 필요한 패키지들 불러오기

In [1]:
import os,warnings
warnings.filterwarnings('ignore')
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

import imageio

해당 실습을 진행하기 위한 주소를 지정해보도록 하겠다. 해당 폴더에는 생성한 MNIST데이터를 저장하려한다.

In [2]:
ROOT_DIR = "D:\\PROJECT\\2019_Education_DL\\GAN"
os.chdir(ROOT_DIR)

추가적으로 GAN을 이용할 때, CPU를 이용해 생성하는 것보다 GPU를 이용한다면 빠르게 생성과 학습이 가능하기 때문에, GPU와 PyTorch가 연동이 되는지 우선적으로 확인한다. True값이 출력된다면, PyTorch로 연산을 진행 할 때, GPU가 사용가능하다는 것이다.

In [3]:
torch.cuda.is_available()

True

In [4]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

#### Generator Class를 생성하는 코드

PyTorch의 인공 신경망 모형(Neural Network)은 클래스를 통해 생성한다.

In [5]:
class Generator(torch.nn.Module):
    def __init__(self, input_dim, num_filters, output_dim):
        super(Generator, self).__init__()

        
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            
            if i == 0:
                deconv = torch.nn.ConvTranspose2d(input_dim, num_filters[i], kernel_size=4, stride=1, padding=0)
            else:
                deconv = torch.nn.ConvTranspose2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

            deconv_name = 'deconv' + str(i + 1)
            self.hidden_layer.add_module(deconv_name, deconv)

            
            torch.nn.init.normal(deconv.weight, mean=0.0, std=0.02)
            torch.nn.init.constant(deconv.bias, 0.0)

            
            bn_name = 'bn' + str(i + 1)
            self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

            
            act_name = 'act' + str(i + 1)
            self.hidden_layer.add_module(act_name, torch.nn.ReLU())

        
        self.output_layer = torch.nn.Sequential()
        
        out = torch.nn.ConvTranspose2d(num_filters[i], output_dim, kernel_size=4, stride=2, padding=1)
        self.output_layer.add_module('out', out)
        
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        
        self.output_layer.add_module('act', torch.nn.Tanh())

    def forward(self, x):
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

#### Discriminator Class를 생성하는 코드

In [6]:
class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, num_filters, output_dim):
        super(Discriminator, self).__init__()

        
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            
            if i == 0:
                conv = torch.nn.Conv2d(input_dim, num_filters[i], kernel_size=4, stride=2, padding=1)
            else:
                conv = torch.nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

            conv_name = 'conv' + str(i + 1)
            self.hidden_layer.add_module(conv_name, conv)

            
            torch.nn.init.normal(conv.weight, mean=0.0, std=0.02)
            torch.nn.init.constant(conv.bias, 0.0)

            
            if i != 0:
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

            
            act_name = 'act' + str(i + 1)
            self.hidden_layer.add_module(act_name, torch.nn.LeakyReLU(0.2))

        
        self.output_layer = torch.nn.Sequential()
        
        out = torch.nn.Conv2d(num_filters[i], output_dim, kernel_size=4, stride=1, padding=0)
        self.output_layer.add_module('out', out)
        
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        
        self.output_layer.add_module('act', torch.nn.Sigmoid())

    def forward(self, x):
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

weight 설정

In [7]:
def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

Latent Space 결정하기

In [8]:
fixed_z_ = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)
fixed_z_ = Variable(fixed_z_.cuda())

Loss 확인하는 그림 그리는 함수

In [9]:
def plot_loss(d_losses, g_losses, num_epoch, save=False, save_dir='./MNIST_DCGAN_results/', show=False):
    fig, ax = plt.subplots()
    ax.set_xlim(0, num_epochs)
    ax.set_ylim(0, max(np.max(g_losses), np.max(d_losses))*1.1)
    plt.xlabel('Epoch {0}'.format(num_epoch + 1))
    plt.ylabel('Loss values')
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.legend()

    # save figure
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'MNIST_DCGAN_losses_epoch_{:d}'.format(num_epoch + 1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()


생성 결과를 확인하는 그림 그리는 함수

In [10]:
def plot_result(generator, noise, num_epoch, save=False, save_dir='./MNIST_DCGAN_results/', show=False, fig_size=(5, 5)):
    generator.eval()

    noise = Variable(noise.cuda())
    gen_image = generator(noise)
    gen_image = denorm(gen_image)

    generator.train()

    n_rows = np.sqrt(noise.size()[0]).astype(np.int32)
    n_cols = np.sqrt(noise.size()[0]).astype(np.int32)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    for ax, img in zip(axes.flatten(), gen_image):
        ax.axis('off')
        ax.set_adjustable('box-forced')
        ax.imshow(img.cpu().data.view(image_size, image_size).numpy(), cmap='gray', aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch+1)
    fig.text(0.5, 0.04, title, ha='center')

    # save figure
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'MNIST_DCGAN_epoch_{:d}'.format(num_epoch+1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()


Hyperparameter 지정하기

In [12]:
image_size = 64
G_input_dim = 100
G_output_dim = 1
D_input_dim = 1
D_output_dim = 1
num_filters = [1024, 512, 256, 128]
# num_filters = [256, 128]

In [13]:
learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 512
num_epochs = 20
DATA_DIR = '../Data/MNIST_data/'
SAVE_DIR = 'MNIST_DCGAN_results/'

In [14]:
transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, ), std=(0.5, ))])

MNIST Data Download 및 Kernel에 업로드

In [15]:
mnist_data = dsets.MNIST(root=DATA_DIR,
                         train=True,
                         transform=transform,
                         download=True)


In [16]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_data,
                                          batch_size=batch_size,
                                          shuffle=True)

#### Generator/ Discriminator 생성 및 초기화

In [17]:
G = Generator(G_input_dim, num_filters, G_output_dim)
D = Discriminator(D_input_dim, num_filters[::-1], D_output_dim)

In [18]:
G.cuda()

Generator(
  (hidden_layer): Sequential(
    (deconv1): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
    (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU()
    (deconv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU()
    (deconv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU()
    (deconv4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act4): ReLU()
  )
  (output_layer): Sequential(
    (out): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (act): Tanh()
  )
)

In [19]:
D.cuda()

Discriminator(
  (hidden_layer): Sequential(
    (conv1): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (act1): LeakyReLU(negative_slope=0.2)
    (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): LeakyReLU(negative_slope=0.2)
    (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): LeakyReLU(negative_slope=0.2)
    (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act4): LeakyReLU(negative_slope=0.2)
  )
  (output_layer): Sequential(
    (out): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1))
    (act): Sigmoid()
  )
)

In [20]:
criterion = torch.nn.BCELoss()

In [21]:
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=betas)

In [22]:
if not os.path.isdir('MNIST_DCGAN_results'):
    os.mkdir('MNIST_DCGAN_results')
if not os.path.isdir('MNIST_DCGAN_results/Random_results'):
    os.mkdir('MNIST_DCGAN_results/Random_results')
if not os.path.isdir('MNIST_DCGAN_results/Fixed_results'):
    os.mkdir('MNIST_DCGAN_results/Fixed_results')

In [23]:
D_avg_losses = []
G_avg_losses = []


In [24]:
num_test_samples = 5*5
fixed_noise = torch.randn(num_test_samples, G_input_dim).view(-1, G_input_dim, 1, 1)

In [ ]:
for epoch in range(num_epochs):
    D_losses = []
    G_losses = []

    # minibatch training
    for i, (images, _) in enumerate(data_loader):

        # image data
        mini_batch = images.size()[0]
        
        # 만약 GAN(인공신경망 구조만 사용) 한다면 Input의 변화 필요
        # x_ = x_.view(-1, image_size*image_size)
        
        x_ = Variable(images.cuda())

        # labels
        y_real_ = Variable(torch.ones(mini_batch).cuda())
        y_fake_ = Variable(torch.zeros(mini_batch).cuda())

        

        D_real_decision = D(x_).squeeze()
        # print(D_real_decision, y_real_)
        D_real_loss = criterion(D_real_decision, y_real_)

        
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())
        gen_image = G(z_)

        D_fake_decision = D(gen_image).squeeze()
        D_fake_loss = criterion(D_fake_decision, y_fake_)

        
        D_loss = D_real_loss + D_fake_loss
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())
        gen_image = G(z_)

        D_fake_decision = D(gen_image).squeeze()
        G_loss = criterion(D_fake_decision, y_real_)

        
        D.zero_grad()
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        
        D_losses.append(D_loss.item())
        G_losses.append(G_loss.item())

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(data_loader), D_loss.item(), G_loss.item()))

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    plot_loss(D_avg_losses, G_avg_losses, epoch, save=True)

    
    plot_result(G, fixed_noise, epoch, save=True, fig_size=(5, 5))

Epoch [1/20], Step [1/118], D_loss: 2.2669, G_loss: 2.8212
Epoch [1/20], Step [2/118], D_loss: 3.7245, G_loss: 4.2486
Epoch [1/20], Step [3/118], D_loss: 1.3690, G_loss: 8.2604
Epoch [1/20], Step [4/118], D_loss: 0.1246, G_loss: 7.8901
Epoch [1/20], Step [5/118], D_loss: 0.3469, G_loss: 8.2984
Epoch [1/20], Step [6/118], D_loss: 0.2251, G_loss: 8.4135
Epoch [1/20], Step [7/118], D_loss: 0.1850, G_loss: 8.9105
Epoch [1/20], Step [8/118], D_loss: 0.0806, G_loss: 8.1014
Epoch [1/20], Step [9/118], D_loss: 0.1701, G_loss: 9.9637
Epoch [1/20], Step [10/118], D_loss: 0.0520, G_loss: 8.7435
Epoch [1/20], Step [11/118], D_loss: 0.2666, G_loss: 13.9620
Epoch [1/20], Step [12/118], D_loss: 0.0675, G_loss: 12.4994
Epoch [1/20], Step [13/118], D_loss: 0.0422, G_loss: 7.3212
Epoch [1/20], Step [14/118], D_loss: 1.3958, G_loss: 24.7287
Epoch [1/20], Step [15/118], D_loss: 1.6080, G_loss: 22.6364
Epoch [1/20], Step [16/118], D_loss: 0.0020, G_loss: 10.2973
